<img src="images/array_print.png" alt="Array Print" align="right"/>

# Array Print

<code>Array Print</code> takes a <code>.csv</code> containing a MITOMI-assayable library in a tabular format and returns a Scienion-compatible <code>.fld</code> file and a csv record of the array layout.

<br>

<div style="width:70px; height:auto; float:left; display:inline">Author:</div>
<div>Micah Olivas</div>
<div style="width:70px; height:auto; float:left; display:inline">Created:</div>
<div>Sept 27, 2021</div>
<div style="width:70px; height:auto; float:left; display:inline">Updated:</div>
<div>May 23, 2022</div>

In [8]:
import os
from ipyfilechooser import FileChooser

import bin.array_print_core as ap

## Import `.csv` containing samples in tabular format

In [9]:
# take user input and get current working directory
user_binning = str(input('Does your print require catalytic binning (specific to HT-MEK)? (yes or no)'))
cwd = os.getcwd()

if user_binning == 'yes':
    print('Select `.csv` containing library positions in tabular format')
    library_csv = FileChooser(cwd)
    display(library_csv)
    print('Select `.csv` containing catalytic bins of mutants in plate')
    fc_catalytic_bins = FileChooser(cwd)
    display(fc_catalytic_bins)
    # bin_desc = pd.read_csv('etc')
    
elif user_binning == 'no':
    print('Select `.csv` containing library positions in tabular format')
    library_csv = FileChooser(cwd)
    display(library_csv)

Select `.csv` containing library positions in tabular format


FileChooser(path='/Users/duncanmuir/code/pinney-lab/Array-Print', filename='', title='', show_hidden=False, se…

In [10]:
library_df, column_names, library_members = ap.csv_to_df(library_csv)

plate_number plate_position                   member_name  catalytic_bin
0              1             F1            EK_WT_Q306_Plasmid              1
1              1             F2           D4A_WT_Q306_Plasmid              1
2              1             F3         EK_C145A_Q306_Plasmid              1
3              1             F4        D4A_C145A_Q306_Plasmid              1
4              1             F5           EK_WT_Q306A_Plasmid              1
5              1             F6          D4A_WT_Q306A_Plasmid              1
6              1             G1      EK_WT_Q306_CrudeAmplicon              1
7              1             G2     D4A_WT_Q306_CrudeAmplicon              1
8              1             G3   EK_C145A_Q306_CrudeAmplicon              1
9              1             G4  D4A_C145A_Q306_CrudeAmplicon              1
10             1             G5     EK_WT_Q306A_CrudeAmplicon              1
11             1             G6    D4A_WT_Q306A_CrudeAmplicon              1
12             1             H1      EK_WT_Q306_CleanAmplicon              1
13             1             H2     D4A_WT_Q306_CleanAmplicon              1
14             1             H3   EK_C145A_Q306_CleanAmplicon              1
15             1             H4  D4A_C145A_Q306_CleanAmplicon              1
16             1             H5     EK_WT_Q306A_CleanAmplicon              1
17             1             H6    D4A_WT_Q306A_CleanAmplicon              1

## Declare print dimensions
This script will maximize the number of sample replicates that appear in the final print given the size of your library, the print dimensions, and any space occuppied by skipped intervening rows. Additionally, you must declare how many blocks appear on your device and how many **empty columns**, if any, you'd like in your print.

In [11]:
# Number of columns and rows on device
total_columns = 32
total_rows = 56
empty_columns = 0
skip_rows = 'n'

library_members, library_size, empty_rows, columns, rows, replicates = ap.count_replicates(library_df, 
                                                                                           total_columns, 
                                                                                           total_rows, 
                                                                                           empty_columns, 
                                                                                           skip_rows)

Library contains 18 members. Will array 99 replicates per library member.


# Name file and generate array
**Change** the name of the file to include a description of your library and today's date. **BE AWARE THAT** each time you execute this cell, **the order of the array changes** and a new, timestamped `.csv` file is written to a project folder in the working directory.

In [12]:
filename = '20230309_mpro_library_2'
print_df, project_path = ap.generate_array(filename, library_df, total_columns, total_rows, skip_rows, column_names)

     0    1    2    3    4    5    6    7    8    9   ...   22   23   24   25  \
0   1G6  1H3  1H3  1G3  1G6  1F2  1G1  1H1  1F3  1G5  ...  1F3  1G5  1H6  1H3   
1   1G1  1H3  1G5  1H4  1F6  1H1  1H5  1H3  1G4  1F3  ...  1H5  1F1  1F3  1H1   
2   1H4  1F5  1G3  1F4  1F6  1G2  1F5  1H6  1H6  1F3  ...  1H6  1F2  1G1  1H4   
3   1F6  1F4  1H6  1H6  1F3  1H5  1H4  1F2  1H2  1G5  ...  1F4  1G2  1H6  1H6   
4   1F3  1G2  1F4  1G6  1H4  1G5  1H6  1F6  1H3  1H2  ...  1F1  1G1  1H3  1F3   
5   1F2  1F5  1H1  1F5  1F5  1F1  1F5  1F1  1G6  1G4  ...  1F5  1F6  1G4  1H2   
6   1H5  1F3  1F5  1F3  1F2  1G3  1H4  1H4  1H2  1G2  ...  1F5  1H4  1F6  1H1   
7   1F4  1G2  1F2  1G6  1F6  1G3  1F4  1F2  1H1  1F3  ...  1H4  1F5  1G5  1F2   
8   1H1  1F6  1G1  1H1  1G3  1H3  1H6  1H6  1F1  1F3  ...  1H3  1G4  1G1  1G3   
9   1G2  1G3  1H5  1G6  1G1  1G5  1F4  1G6  1G2  1H5  ...  1G1  1H2  1G4  1G4   
10  1F1  1H2  1H3  1F3  1F1  1H6  1F1  1F6  1H2  1H5  ...  1H2  1H2  1G3  1H4   
11  1G5  1G3  1G6  1G1  1H1 

## Output field file-formatted position list
Finally, the following cell writes a `.txt` file containing the array position list to the project folder. This text can be copied into a Scienion-compattible `.fld` file for printing. Note that a new, timestamped `.txt` file is written each time you execute this cell. 

In [13]:
ap.display_fld(print_df, total_columns, total_rows)
ap.write_fld(project_path, filename, print_df, total_columns, total_rows)

1/1	1G6,	1,
1/2	1H3,	1,
1/3	1H3,	1,
1/4	1G3,	1,
1/5	1G6,	1,
1/6	1F2,	1,
1/7	1G1,	1,
1/8	1H1,	1,
1/9	1F3,	1,
1/10	1G5,	1,
1/11	1G2,	1,
1/12	1F6,	1,
1/13	1H5,	1,
1/14	1F6,	1,
1/15	1H1,	1,
1/16	1F5,	1,
1/17	1F2,	1,
1/18	1F5,	1,
1/19	1G4,	1,
1/20	1G6,	1,
1/21	1F5,	1,
1/22	1G6,	1,
1/23	1F3,	1,
1/24	1G5,	1,
1/25	1H6,	1,
1/26	1H3,	1,
1/27	1H1,	1,
1/28	1G4,	1,
1/29	1F2,	1,
1/30	1F1,	1,
1/31	1G1,	1,
1/32	1H1,	1,
2/1	1G1,	1,
2/2	1H3,	1,
2/3	1G5,	1,
2/4	1H4,	1,
2/5	1F6,	1,
2/6	1H1,	1,
2/7	1H5,	1,
2/8	1H3,	1,
2/9	1G4,	1,
2/10	1F3,	1,
2/11	1F6,	1,
2/12	1F1,	1,
2/13	1F4,	1,
2/14	1H3,	1,
2/15	1G6,	1,
2/16	1G3,	1,
2/17	1F5,	1,
2/18	1G5,	1,
2/19	1G5,	1,
2/20	1H5,	1,
2/21	1F4,	1,
2/22	1H4,	1,
2/23	1H5,	1,
2/24	1F1,	1,
2/25	1F3,	1,
2/26	1H1,	1,
2/27	1F1,	1,
2/28	1H4,	1,
2/29	1G6,	1,
2/30	1F4,	1,
2/31	1H4,	1,
2/32	1H1,	1,
3/1	1H4,	1,
3/2	1F5,	1,
3/3	1G3,	1,
3/4	1F4,	1,
3/5	1F6,	1,
3/6	1G2,	1,
3/7	1F5,	1,
3/8	1H6,	1,
3/9	1H6,	1,
3/10	1F3,	1,
3/11	1F5,	1,
3/12	1G3,	1,
3/13	1G3,	1,
3/14	1G6,	1,
3/15	1G3,	1,
